### Extracción de capas de interés de SIOSE
#### (Solar, Cultivos y Cultivos arbóreos)

##### Instalar paquetes no nativos

In [ ]:
!pip install geopandas pyproj

In [ ]:
!pip install fiona

##### Importar paquetes

In [1]:
import os
import glob
import geopandas as gpd
from pyproj import CRS
import zipfile
import fiona


##### Listar todos los archivos de SIOSE, formato .gpkg para cada provincia

In [2]:
# Directorio raíz
directorio_raiz = "spatial_data/"

# Lista para almacenar los archivos .gpkg
gpkg_files = []

# Recorrer recursivamente el directorio raíz y sus subdirectorios
for directorio_actual, _, archivos in os.walk(directorio_raiz):
    # Filtrar los archivos .gpkg y agregarlos a la lista
    gpkg_files += [os.path.join(directorio_actual, archivo) for archivo in archivos if archivo.endswith(".gpkg")]

# Imprimir la lista de archivos .gpkg
print(gpkg_files)

['spatial_data/01_ALAVA.gpkg', 'spatial_data/03_ALICANTE.gpkg', 'spatial_data/04_ALMERIA.gpkg', 'spatial_data/05_AVILA.gpkg', 'spatial_data/06_BADAJOZ.gpkg', 'spatial_data/08_BARCELONA.gpkg', 'spatial_data/09_BURGOS.gpkg', 'spatial_data/10_CACERES.gpkg', 'spatial_data/12_CASTELLON.gpkg', 'spatial_data/14_CORDOBA.gpkg', 'spatial_data/15_CORUÑA.gpkg', 'spatial_data/17_GIRONA.gpkg', 'spatial_data/18_GRANADA.gpkg', 'spatial_data/20_GUIPUZCOA.gpkg', 'spatial_data/21_HUELVA.gpkg', 'spatial_data/22_HUESCA.gpkg', 'spatial_data/23_JAEN.gpkg', 'spatial_data/24_LEON.gpkg', 'spatial_data/25_LLEIDA.gpkg', 'spatial_data/26_RIOJA.gpkg', 'spatial_data/27_LUGO.gpkg', 'spatial_data/28_MADRID.gpkg', 'spatial_data/29_MALAGA.gpkg', 'spatial_data/30_MURCIA.gpkg', 'spatial_data/31_NAVARRA.gpkg', 'spatial_data/32_OURENSE.gpkg', 'spatial_data/33_ASTURIAS.gpkg', 'spatial_data/34_PALENCIA.gpkg', 'spatial_data/37_SALAMANCA.gpkg', 'spatial_data/39_CANTABRIA.gpkg', 'spatial_data/40_SEGOVIA.gpkg', 'spatial_data/41_S

##### Extracción polígonos solares

In [4]:
# Función para extraer la capa USOS y el uso 'Solar'
def extract_USOS(file_path):
    # Listar todas las capas
    layers = fiona.listlayers(file_path)
    # Seleccionar el nombre de capa que termine por '_T_USOS'
    layer_name = [layer for layer in layers if layer.endswith('_T_USOS')][0]
    # Leer la capa
    layer = gpd.read_file(file_path, layer=layer_name)
    # Extraer uso 'Solar'
    layer = layer[layer['ID_USO_MAX'].isin([2442])]
    return layer

In [5]:
# Extraer capas 
solar_polygons = [extract_USOS(file) for file in gpkg_files]

In [ ]:
# Proyectar a EPSG:25830
solar_polygons = [capa.to_crs(25830) for capa in solar_polygons]

In [ ]:
# Combinar capas en un único objeto
solar = solar_polygons[0]
for capa in solar_polygons[1:]:
    solar = solar.append(capa, ignore_index=True)

In [ ]:
# Guardar
solar.to_file("spatial_data/final_polys/solar.shp")

In [ ]:
# NO SE SI ES ESTO O LA ANTERIOR CELDA TENGO QUE PROBAR
gpd.GeoDataFrame(pd.concat(capas)).to_file("spatial_data/final_polys/solar.shp")

##### Extracción polígonos invernaderos

In [8]:
# Función para extraer la capa COBERTURAS y el uso 'Cultivos' (invernaderos)
def extract_cultivos(file_path):
    # Listar todas las capas
    layers = fiona.listlayers(file_path)
    # Seleccionar el nombre de capa que termine por '_T_COMBINADA'
    layer_name = [layer for layer in layers if layer.endswith('_T_COMBINADA')][0]
    # Leer la capa
    layer = gpd.read_file(file_path, layer=layer_name)
    # Extraer uso 'Producción agrícola comercial'
    layer = layer[layer['COBERTURA_DESC'].isin(['Cultivos'])]
    return layer

In [9]:
# Extraer capas 
greenhouse_polygons = [extract_cultivos(file) for file in gpkg_files]

In [ ]:
# Proyectar a EPSG:25830
greenhouse_polygons = [capa.to_crs(25830) for capa in greenhouse_polygons]

In [ ]:
# Combinar capas en un único objeto
greenhouse = greenhouse_polygons[0]
for capa in greenhouse_polygons[1:]:
    greenhouse = greenhouse.append(capa, ignore_index=True)